In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
running_order.head()

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.743671,1.863840,2.033659,0.084909,0.100000,2.110,0.100000
1,1,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,2,5.7,joana wälti,2,Joana Wälti,2.0,2.102,0.124,6.882701,1.809328,2.053610,0.122141,0.122141,2.102,0.122141
2,2,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,3,7.3,anna kindlundh,3,Anna Kindlundh,6.0,2.077,0.120,6.107582,1.726928,2.056589,0.164830,0.164830,2.077,0.120000
3,3,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,4,7.9,jeanette jönsson hellstadius,4,Jeanette Jönsson Hellstadius,3.0,2.041,0.134,7.474857,1.845341,2.089668,0.122164,0.122164,2.041,0.134000
4,4,30,Suunta Jyväskylä 1,Suunta Jyväskylä,FIN,1,6.0,veera klemettinen,1,Veera Klemettinen,6.0,2.064,0.106,7.198831,1.873431,2.050207,0.088388,0.100000,2.064,0.106000


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,8,Södertälje Nykvarn Orientering 1,1,6.0,Fiona Bunn,2.110,0.100000
1,8,Södertälje Nykvarn Orientering 1,2,5.7,Joana Wälti,2.102,0.122141
2,8,Södertälje Nykvarn Orientering 1,3,7.3,Anna Kindlundh,2.077,0.120000
3,8,Södertälje Nykvarn Orientering 1,4,7.9,Jeanette Jönsson Hellstadius,2.041,0.134000
4,30,Suunta Jyväskylä 1,1,6.0,Veera Klemettinen,2.064,0.106000


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                    \
leg                                     1                                 2   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   
41              Lahden Suunnistajat -37 1         Lahden Suunnistajat -37 1   
52                            Byåsen IL 1                       Byåsen IL 1   
58                        Northern Wind 1                   Northern Wind 1   

                                                                             \
leg                                     3                                 4   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   
41              Lahden Suunnistajat -37 1         Lahden Suunnistajat -37 1   
52                            Byåsen IL 1                       Byåsen IL 1   
58                        Northern Wind 1                   Northern Wind 1   

        dist                                  name                        \
leg        1    2    3    4                      1                     2   
team_id                                                                    
8        6.0  5.7  7.3  7.9             Fiona Bunn           Joana Wälti   
30       6.0  5.7  7.3  7.9      Veera Klemettinen      Elisa Viholainen   
41       6.0  5.7  7.3  7.9         Laura Ramstein      Lotta Kirvesmies   
52       6.0  5.7  7.3  7.9          Ingeborg Eide          Emma Arnesen   
58       6.0  5.7  7.3  7.9  Anastassia Grigorieva  Elizaveta Rastegaeva   

                                                            log_mean         \
leg                        3                             4         1      2   
team_id                                                                       
8             Anna Kindlundh  Jeanette Jönsson Hellstadius  2.110000  2.102   
30        Barbora Chaloupska             Kaisa Klemettinen  2.064000  2.061   
41         Anna-Kaisa Anjala                  Minna Kauppi  1.973000  1.968   
52                Göril Eide                 Jenny Persson  2.069000  2.023   
58       Natalia Kondrashova              Svetlana Raikova  2.235949  2.036   

                      log_std                             
leg          3      4       1         2         3      4  
team_id                                                   
8        2.077  2.041   0.100  0.122141  0.120000  0.134  
30       1.984  2.050   0.106  0.122000  0.140614  0.138  
41       2.024  2.025   0.116  0.097000  0.147421  0.109  
52       2.128  2.295   0.095  0.102000  0.121845  0.206  
58       2.215  1.938   0.100  0.136361  0.142952  0.099

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(547, 20)

(537, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict, **fin_sum_log_means_dict, **fin_sum_log_stds_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,fin_mean_3,fin_mean_4,fin_sum_log_mean_1,fin_sum_log_mean_2,fin_sum_log_mean_3,fin_sum_log_mean_4,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,40.483222,82.404882,135.330532,191.172872,60.326404,112.387746,176.156658,243.655692,49.481670,96.394480,...,155.321969,216.652803,3.901529,4.568527,5.043235,5.376422,0.100795,0.078969,0.067359,0.061262
30,38.348779,78.973246,127.346561,183.264861,58.485894,108.231315,168.686634,236.678984,47.204311,92.434679,...,146.559462,208.477665,3.856625,4.526871,4.984798,5.337678,0.106680,0.080233,0.072509,0.065610
41,34.611571,72.400360,121.079587,177.692876,54.226689,97.943196,162.534626,226.458803,43.207034,84.127548,...,140.320052,200.518614,3.765649,4.432754,4.941117,5.298977,0.114389,0.076372,0.074893,0.062083
52,39.639752,79.271753,135.001442,199.017694,57.117995,103.882053,173.264482,276.220696,47.530390,90.816688,...,152.731752,232.998506,3.861295,4.508807,5.026630,5.447540,0.094720,0.069518,0.064039,0.083318
58,46.351106,85.418124,144.584940,197.412022,68.392025,117.739508,194.092589,251.909644,56.157238,100.042754,...,167.926267,223.104859,4.027996,4.606642,5.120678,5.405734,0.098840,0.081780,0.075459,0.061778


In [8]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()

,team_id,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,...,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,...,Anna Kindlundh,Jeanette Jönsson Hellstadius,2.110000,2.102000,2.077000,2.041000,0.100000,0.122141,0.120000,0.134000
1,30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,...,Barbora Chaloupska,Kaisa Klemettinen,2.064000,2.061000,1.984000,2.050000,0.106000,0.122000,0.140614,0.138000
2,41,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,6.0,5.7,7.3,7.9,Laura Ramstein,...,Anna-Kaisa Anjala,Minna Kauppi,1.973000,1.968000,2.024000,2.025000,0.116000,0.097000,0.147421,0.109000
3,52,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,6.0,5.7,7.3,7.9,Ingeborg Eide,...,Göril Eide,Jenny Persson,2.069000,2.023000,2.128000,2.295000,0.095000,0.102000,0.121845,0.206000
4,58,Northern Wind 1,Northern Wind 1,Northern Wind 1,Northern Wind 1,6.0,5.7,7.3,7.9,Anastassia Grigorieva,...,Natalia Kondrashova,Svetlana Raikova,2.235949,2.036000,2.215000,1.938000,0.100000,0.136361,0.142952,0.099000
5,68,Asker SK 1,Asker SK 1,Asker SK 1,Asker SK 1,6.0,5.7,7.3,7.9,Synne Strand,...,Anna Öfsthus Gravir,Marit Isachsen,2.139000,2.121000,2.274000,2.307000,0.130934,0.236187,0.182309,0.235000
6,78,Lahden Suunnistajat -37 2,Lahden Suunnistajat -37 2,Lahden Suunnistajat -37 2,Lahden Suunnistajat -37 2,6.0,5.7,7.3,7.9,Tytti Kirvesmies,...,Anniina Taipale,Saara Kukkonen,1.986000,2.126671,2.276000,1.997000,0.105000,0.164399,0.164521,0.125000
7,80,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,6.0,5.7,7.3,7.9,Mari Eidsmo,...,Dina Reitan,Vera Melkild,2.081351,2.068919,2.090712,2.102000,0.167446,0.142361,0.159152,0.131051
8,83,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,6.0,5.7,7.3,7.9,Laura King,...,Sofia Wigle,Anna Ek Larsson,2.070204,2.055800,2.060000,2.088466,0.137841,0.140509,0.160034,0.134402
9,88,Nyköpings OK 1,Nyköpings OK 1,Nyköpings OK 1,Nyköpings OK 1,6.0,5.7,7.3,7.9,Hylander Kristin,...,Hylander Rebecca,Nilsson Signe,2.119911,2.128447,2.142814,2.190142,0.141507,0.155136,0.169570,0.136153


In [9]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [10]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,fin_mean_3,fin_mean_4,fin_sum_log_mean_1,fin_sum_log_mean_2,fin_sum_log_mean_3,fin_sum_log_mean_4,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,...,155.321969,216.652803,3.901529,4.568527,5.043235,5.376422,0.100795,0.078969,0.067359,0.061262
30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,Elisa Viholainen,...,146.559462,208.477665,3.856625,4.526871,4.984798,5.337678,0.106680,0.080233,0.072509,0.065610
41,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,Lahden Suunnistajat -37 1,6.0,5.7,7.3,7.9,Laura Ramstein,Lotta Kirvesmies,...,140.320052,200.518614,3.765649,4.432754,4.941117,5.298977,0.114389,0.076372,0.074893,0.062083
52,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,Byåsen IL 1,6.0,5.7,7.3,7.9,Ingeborg Eide,Emma Arnesen,...,152.731752,232.998506,3.861295,4.508807,5.026630,5.447540,0.094720,0.069518,0.064039,0.083318
58,Northern Wind 1,Northern Wind 1,Northern Wind 1,Northern Wind 1,6.0,5.7,7.3,7.9,Anastassia Grigorieva,Elizaveta Rastegaeva,...,167.926267,223.104859,4.027996,4.606642,5.120678,5.405734,0.098840,0.081780,0.075459,0.061778


In [11]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
30      1                  Suunta Jyväskylä 1   6.0   
        2                  Suunta Jyväskylä 1   5.7   
        3                  Suunta Jyväskylä 1   7.3   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.483222   
        2                     Joana Wälti     2.102  0.122141    82.404882   
        3                  Anna Kindlundh     2.077  0.120000   135.330532   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   191.172872   
30      1               Veera Klemettinen     2.064  0.106000    38.348779   
        2                Elisa Viholainen     2.061  0.122000    78.973246   
        3              Barbora Chaloupska     1.984  0.140614   127.346561   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.326404   49.481670   49.729928          3.901529   
        2    112.387746   96.394480   96.702721          4.568527   
        3    176.156658  155.034719  155.321969          5.043235   
        4    243.655692  216.431774  216.652803          5.376422   
30      1     58.485894   47.204311   47.575946          3.856625   
        2    108.231315   92.434679   92.767281          4.526871   
        3    168.686634  146.212479  146.559462          4.984798   

             fin_sum_log_std  
team_id leg                   
8       1           0.100795  
        2           0.078969  
        3           0.067359  
        4           0.061262  
30      1           0.106680  
        2           0.080233  
        3           0.072509

In [12]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95"]]

,,name,fin_start95,fin_median,fin_end95
team_id,leg,,,,


In [13]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates.head()

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
30      1                  Suunta Jyväskylä 1   6.0   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.483222   
        2                     Joana Wälti     2.102  0.122141    82.404882   
        3                  Anna Kindlundh     2.077  0.120000   135.330532   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   191.172872   
30      1               Veera Klemettinen     2.064  0.106000    38.348779   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.326404   49.481670   49.729928          3.901529   
        2    112.387746   96.394480   96.702721          4.568527   
        3    176.156658  155.034719  155.321969          5.043235   
        4    243.655692  216.431774  216.652803          5.376422   
30      1     58.485894   47.204311   47.575946          3.856625   

             fin_sum_log_std               fin_time_median  \
team_id leg                                                  
8       1           0.100795 2019-06-15 14:49:28.900215149   
        2           0.078969 2019-06-15 15:36:23.668774843   
        3           0.067359 2019-06-15 16:35:02.083133221   
        4           0.061262 2019-06-15 17:36:25.906447172   
30      1           0.106680 2019-06-15 14:47:12.258676766   

                         fin_time_start95                fin_time_end95  
team_id leg                                                              
8       1   2019-06-15 14:40:28.993305445 2019-06-15 15:00:19.584246874  
        2   2019-06-15 15:22:24.292928219 2019-06-15 15:52:23.264744519  
        3   2019-06-15 16:15:19.831943512 2019-06-15 16:56:09.399472713  
        4   2019-06-15 17:11:10.372349501 2019-06-15 18:03:39.341535568  
30      1   2019-06-15 14:38:20.926741838 2019-06-15 14:58:29.153642178

In [14]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["fin_time_median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["fin_time_start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["fin_time_end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")
for_print = for_print[['team',
 'leg',
 'name', 
 'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_print[for_print['team'].str.contains("Reak")]

,team,leg,name,fin_time_start95,fin_time_end95,fin_time_median
team_id,,,,,,


In [15]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median
248,341,1,Rasti-Jyry 1,Jola Maisala,2.197000,0.117000,3.988956,0.116814,2019-06-15 11:42:58.254418135+00:00,2019-06-15 12:08:03.663071871+00:00,2019-06-15 11:54:01.319119453+00:00
249,341,2,Rasti-Jyry 1,Kaija Rantoja,2.364000,0.140000,4.746236,0.092862,2019-06-15 12:36:18.027906418+00:00,2019-06-15 13:18:35.893737793+00:00,2019-06-15 12:55:07.466194868+00:00
250,341,3,Rasti-Jyry 1,Jenni Castren,2.395000,0.152000,5.278199,0.083303,2019-06-15 13:46:56.379363775+00:00,2019-06-15 14:51:16.748057604+00:00,2019-06-15 14:15:53.806812286+00:00
251,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.558000,0.178309,5.701946,0.081669,2019-06-15 15:15:56.134843588+00:00,2019-06-15 16:52:31.322566509+00:00,2019-06-15 15:59:01.936457396+00:00
448,528,1,Rasti-Jyry 2,Päivi Taponen,2.418000,0.163000,4.208552,0.161960,2019-06-15 11:48:57.896326303+00:00,2019-06-15 12:32:17.431728363+00:00,2019-06-15 12:07:14.211797714+00:00
449,528,2,Rasti-Jyry 2,Miia Hovi,2.229000,0.141000,4.794774,0.110402,2019-06-15 12:37:22.193197966+00:00,2019-06-15 13:29:54.247263432+00:00,2019-06-15 13:00:51.734873533+00:00
450,528,3,Rasti-Jyry 2,Mervi Luurila,2.459000,0.191317,5.336075,0.102109,2019-06-15 13:49:13.110285997+00:00,2019-06-15 15:13:42.206261396+00:00,2019-06-15 14:27:45.519604682+00:00
451,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.371000,0.116000,5.679569,0.080317,2019-06-15 15:09:43.534888744+00:00,2019-06-15 16:42:46.060115814+00:00,2019-06-15 15:52:45.794512510+00:00
660,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.144432,4.375605,0.143511,2019-06-15 12:00:10.627411604+00:00,2019-06-15 12:44:54.162104130+00:00,2019-06-15 12:19:21.902783632+00:00
661,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.617000,0.174000,5.065391,0.112758,2019-06-15 13:07:32.048123121+00:00,2019-06-15 14:17:58.457403898+00:00,2019-06-15 13:38:23.636625767+00:00


In [16]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [17]:
np.exp(5.210428700661273)

183.1725675648483

In [18]:
np.exp(5.8877321868)

360.58661334518484